In [1]:
import os

from dotenv import load_dotenv
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-academy"

load_dotenv(dotenv_path="../.env", override=True)

True

## Tracing with @traceable  
`@traceable` デコレーターは、LangSmith Python SDKからトレースをログに記録するためのシンプルな方法です。任意の関数に`@traceable`をデコレーターとして付けるだけで利用できます。

このデコレーターは、関数が呼び出されるたびにランツリー（run tree）を作成し、現在のトレース内に挿入する仕組みです。関数の入力、名前、その他の情報がLangSmithにストリームとして送信されます。関数がエラーを発生させた場合やレスポンスを返した場合、その情報もツリーに追加され、LangSmithにパッチ更新されます。そのため、エラーの原因を特定し、診断することが可能になります。これらはすべてバックグラウンドスレッドで実行されるため、アプリケーションの実行がブロックされることはありません。

In [ ]:
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_PROVIDER = "openai"
MODEL_NAME = "gpt-4o-mini"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(run_type="chain")
def retrieve_documents(question:str):
    return retriever.invoke(question)

def generate_answer(question:str, documents:List[str]):
####




